In [102]:

import sqlite3
from bs4 import BeautifulSoup
import requests
import time

# URLを指定
base_url = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?fw2=&ek=058024180&mt=9999999&cn=9999999&ra=012&et=9999999&shkr1=03&ar=030&bs=040&ct=9999999&shkr3=03&shkr2=03&mb=0&rn=0580&shkr4=03&cb=0.0"  # ここに対象のURLを記載

# requestsでHTMLを取得
response = requests.get(base_url)

# レスポンスの内容を確認
if response.status_code == 200:
    html_data = response.text  # 正常に取得できた場合はHTMLを取得
else:
    raise Exception(f"Failed to fetch the webpage. Status code: {response.status_code}")

# BeautifulSoupでHTMLを解析
soup = BeautifulSoup(html_data, "html.parser")

# SQLiteデータベースに接続または新規作成
conn = sqlite3.connect("properties.db")
cursor = conn.cursor()

# テーブルを作成
cursor.execute("""
CREATE TABLE IF NOT EXISTS properties (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    property_name TEXT,
    built_year INTEGER,
    floor TEXT,
    rent TEXT
)
""")

# データを抽出して格納
for cassette in soup.find_all("div", class_="cassetteitem"):
    # 物件名
    property_name = cassette.find("div", class_="cassetteitem_content-title").get_text(strip=True)
    
    # 築年数
    built_info = cassette.find("li", class_="cassetteitem_detail-col3")
    built_year = built_info.find_all("div")[0].get_text(strip=True).replace("築", "").replace("年", "") if built_info else None
    
    # 部屋情報
    for room_row in cassette.find_all("tr", class_="js-cassette_link"):
        # 階
        floor_td = room_row.find_all("td")[2]
        floor = floor_td.get_text(strip=True) if floor_td else None

        
        # 賃料
        rent_col = room_row.find_all("td")[3]  # <td>要素を取得
        if rent_col:
            rent_li = rent_col.find("li")  # <li>要素を取得
            if rent_li:
                rent_span = rent_li.find("span", class_="cassetteitem_price cassetteitem_price--rent")
                rent = rent_span.get_text(strip=True) if rent_span else None
            else:
                rent = None
        else:
            rent = None
        
        # データをデータベースに挿入
        cursor.execute("""
        INSERT INTO properties (property_name, built_year, floor, rent)
        VALUES (?, ?, ?, ?)
        """, (property_name, built_year, floor, rent))

        time.sleep(1)

# コミットして接続を閉じる
conn.commit()
conn.close()

print("データの保存が完了しました。")



データの保存が完了しました。


In [103]:
import sqlite3

# データベースに接続
conn = sqlite3.connect("properties.db")
cursor = conn.cursor()

# データを取得するクエリを実行
cursor.execute("SELECT * FROM properties")

# 取得したデータを全て表示
rows = cursor.fetchall()
for row in rows:
    print(row)

# 接続を閉じる
conn.close()


(1, 'プライムシティ天台', 8, '1階', '5.1万円')
(2, 'ＪＲ内房線 八幡宿駅 3階建 築4年', 4, '1階', '6.4万円')
(3, 'ＪＲ総武本線 四街道駅 2階建 新築', '新', '1階', '5.1万円')
(4, 'ＪＲ総武本線 四街道駅 2階建 新築', '新', '1階', '5.1万円')
(5, 'リブリ黒砂台', 14, '2階', '6.2万円')
(6, 'コーポスカイＧ', 11, '2階', '7.7万円')
(7, 'コーポスカイＧ', 11, '3階', '8.5万円')
(8, 'コーポスカイＧ', 11, '1階', '8.8万円')
(9, 'ガーデンコート稲毛', 5, '7階', '7.7万円')
(10, 'ＪＲ総武線 千葉駅 4階建 築9年', 9, '1階', '6.7万円')
(11, 'ＪＲ総武線 千葉駅 4階建 築9年', 9, '3階', '7万円')
(12, 'AQUAS Will千葉(アクアス\u3000ウィル)', 9, '1階', '6.7万円')
(13, 'AQUAS Will千葉(アクアス\u3000ウィル)', 9, '3階', '7万円')
(14, 'フォレストツインハウスS', 12, '2階', '5.7万円')
(15, 'ロックウェル轟A', 35, '1階', '4万円')
(16, 'ロックウェル轟A', 35, '2階', '4万円')
(17, 'フェリシテ', 1, '6階', '7.6万円')
(18, 'カーサメイプル千葉中央', 13, '3階', '8.3万円')
(19, 'クラージュグラン院内', '新', '1階', '8.5万円')
(20, 'クラージュグラン院内', '新', '2階', '8.5万円')
(21, 'クラージュグラン院内', '新', '2階', '8.5万円')
(22, 'クラージュグラン院内', '新', '2階', '8.5万円')
(23, 'クラージュグラン院内', '新', '2階', '8.5万円')
(24, 'クラージュグラン院内', '新', '3階', '8.6万円')
(25, 'クラージュグラン院内', '新', '8階', '9.1万円')
(26, 'クラージュグラン院